In [13]:
import numpy as np
import cv2
import os
import random

def generate_cusp_image(img_size=(224, 224), cusp_color=100, bg_color=180, mask_color=255, 
                         add_texture=True, blur=True):
    """Generates an idealised cusp image and corresponding mask with varying ellipse widths and shades of grey."""
    h, w = img_size
    img = np.ones((h, w), dtype=np.uint8) * bg_color
    mask = np.zeros((h, w), dtype=np.uint8)

    # Define cusp shape (U-shape with extended vertical sections)
    center_x, center_y = w // 2, h // 2 + 30
    cusp_radius = random.randint(50, 80)  # Varying width of the ellipse
    thickness = 20

    # Draw the U-shape (cusp) with extended vertical sections
    cv2.ellipse(img, (center_x, center_y), (cusp_radius, cusp_radius//2), 0, 0, 180, cusp_color, thickness)
    cv2.ellipse(mask, (center_x, center_y), (cusp_radius, cusp_radius//2), 0, 0, 180, mask_color, thickness)
    
    # Draw vertical extensions to seamlessly connect with the U-shape
    line_length = 40  # Adjust length for proper connection
    start_y = center_y - cusp_radius//2 + thickness // 2  # Ensure alignment with ellipse
    end_y = start_y - line_length

    # Move the lines downward to connect better with the ellipse
    cv2.line(img, (center_x - cusp_radius, start_y + 5), 
             (center_x - cusp_radius, end_y + 5), cusp_color, thickness)
    cv2.line(img, (center_x + cusp_radius, start_y + 5), 
             (center_x + cusp_radius, end_y + 5), cusp_color, thickness)
    
    cv2.line(mask, (center_x - cusp_radius, start_y + 5), 
             (center_x - cusp_radius, end_y + 5), mask_color, thickness)
    cv2.line(mask, (center_x + cusp_radius, start_y + 5), 
             (center_x + cusp_radius, end_y + 5), mask_color, thickness)

    # Add texture if needed
    if add_texture:
        noise = np.random.normal(0, 10, img.shape).astype(np.uint8)
        img = cv2.add(img, noise)

    # Apply Gaussian blur for smoother transitions
    if blur:
        img = cv2.GaussianBlur(img, (5, 5), 0)

    return img, mask

def save_cusp_images(num_samples=10, output_dir="idealised_cusps"):
    """Generates and saves multiple cusp images and masks with varying shapes and grayscale variations."""
    mask_dir = os.path.join(output_dir, "masks")
    image_dir = os.path.join(output_dir, "images")
    os.makedirs(mask_dir, exist_ok=True)
    os.makedirs(image_dir, exist_ok=True)

    grayscale_options = [(50, 200), (80, 170), (100, 180), (120, 160), (150, 220)]  # Different cusp/bg color pairs
    
    for i in range(num_samples):
        for j, (cusp_color, bg_color) in enumerate(grayscale_options):
            cusp_img, cusp_mask = generate_cusp_image(cusp_color=cusp_color, bg_color=bg_color)
            cv2.imwrite(os.path.join(image_dir, f"cusp_{i}_{j}.png"), cusp_img)
            cv2.imwrite(os.path.join(mask_dir, f"cusp_{i}_{j}.png"), cusp_mask)
    
    print(f"✅ {num_samples * len(grayscale_options)} cusp images and masks saved in '{output_dir}'")

if __name__ == "__main__":
    save_cusp_images(num_samples=10)


✅ 50 cusp images and masks saved in 'idealised_cusps'
